In [1]:
import pandas as pd
import datetime
import json

In [2]:
df_day1 = pd.read_csv('siri.20121106.csv', names=["Timestamp", "Line_ID", "Direction", "JourneyPattern_ID", "Time_Frame", "VehicleJourney_ID", "Operator", "Congestion", "Longitude", "Latitude", "Delay", "Block_ID", "Vehicle_ID", "Stop_ID", "At_Stop"])
df_day2 = pd.read_csv('siri.20121107.csv', names=["Timestamp", "Line_ID", "Direction", "JourneyPattern_ID", "Time_Frame", "VehicleJourney_ID", "Operator", "Congestion", "Longitude", "Latitude", "Delay", "Block_ID", "Vehicle_ID", "Stop_ID", "At_Stop"])
df_day3 = pd.read_csv('siri.20121108.csv', names=["Timestamp", "Line_ID", "Direction", "JourneyPattern_ID", "Time_Frame", "VehicleJourney_ID", "Operator", "Congestion", "Longitude", "Latitude", "Delay", "Block_ID", "Vehicle_ID", "Stop_ID", "At_Stop"])
df_day4 = pd.read_csv('siri.20121109.csv', names=["Timestamp", "Line_ID", "Direction", "JourneyPattern_ID", "Time_Frame", "VehicleJourney_ID", "Operator", "Congestion", "Longitude", "Latitude", "Delay", "Block_ID", "Vehicle_ID", "Stop_ID", "At_Stop"])
df_day5 = pd.read_csv('siri.20121110.csv', names=["Timestamp", "Line_ID", "Direction", "JourneyPattern_ID", "Time_Frame", "VehicleJourney_ID", "Operator", "Congestion", "Longitude", "Latitude", "Delay", "Block_ID", "Vehicle_ID", "Stop_ID", "At_Stop"])
df_day6 = pd.read_csv('siri.20121111.csv', names=["Timestamp", "Line_ID", "Direction", "JourneyPattern_ID", "Time_Frame", "VehicleJourney_ID", "Operator", "Congestion", "Longitude", "Latitude", "Delay", "Block_ID", "Vehicle_ID", "Stop_ID", "At_Stop"])
df_day7 = pd.read_csv('siri.20121112.csv', names=["Timestamp", "Line_ID", "Direction", "JourneyPattern_ID", "Time_Frame", "VehicleJourney_ID", "Operator", "Congestion", "Longitude", "Latitude", "Delay", "Block_ID", "Vehicle_ID", "Stop_ID", "At_Stop"])

C:\Users\pamel\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\pamel\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_day1 = df_day1.loc[(df_day1['At_Stop'] == 1) & (df_day1['JourneyPattern_ID'] != 'null')]
df_day2 = df_day2.loc[(df_day2['At_Stop'] == 1) & (df_day1['JourneyPattern_ID'] != 'null')]
df_day3 = df_day3.loc[(df_day3['At_Stop'] == 1) & (df_day1['JourneyPattern_ID'] != 'null')]
df_day4 = df_day4.loc[(df_day4['At_Stop'] == 1) & (df_day1['JourneyPattern_ID'] != 'null')]
df_day5 = df_day5.loc[(df_day5['At_Stop'] == 1) & (df_day1['JourneyPattern_ID'] != 'null')]
df_day6 = df_day6.loc[(df_day6['At_Stop'] == 1) & (df_day1['JourneyPattern_ID'] != 'null')]
df_day7 = df_day7.loc[(df_day7['At_Stop'] == 1) & (df_day1['JourneyPattern_ID'] != 'null')]

In [4]:
df_day1.shape

(377261, 15)

In [5]:
frames = [df_day1, df_day2, df_day3, df_day4, df_day5, df_day6, df_day7]
df_week = pd.concat(frames)

In [6]:
# journey pattern ids that are equal to null indicate idling bus at depot/terminus
df_week.isnull().sum()

Timestamp             0
Line_ID               0
Direction             0
JourneyPattern_ID    11
Time_Frame            0
VehicleJourney_ID     0
Operator              0
Congestion            0
Longitude             0
Latitude              0
Delay                 0
Block_ID              0
Vehicle_ID            0
Stop_ID               0
At_Stop               0
dtype: int64

In [7]:
df_week.shape

(800861, 15)

In [8]:
df_week.columns

Index(['Timestamp', 'Line_ID', 'Direction', 'JourneyPattern_ID', 'Time_Frame',
       'VehicleJourney_ID', 'Operator', 'Congestion', 'Longitude', 'Latitude',
       'Delay', 'Block_ID', 'Vehicle_ID', 'Stop_ID', 'At_Stop'],
      dtype='object')

In [9]:
# add time and day columns before grouping
day, tm = 'null', 'null'
df_week['Day'], df_week['Time'] = day, tm

In [10]:
df_week = df_week.groupby(['JourneyPattern_ID'])

In [11]:
len(df_week)

471

In [12]:
def convert_timestamp(stamp):
    stamp = (stamp/1000)/1000
    full_dt = datetime.datetime.fromtimestamp(stamp).strftime('%Y-%m-%d %H:%M:%S')
    tm = full_dt[11:]
    dt = full_dt[:10]
    day = datetime.datetime.strptime(dt, '%Y-%m-%d').strftime('%A')
    return tm, day

In [13]:
# Need to loop through the journey groups and group those by vehicle journey id 
# because we don't need all the runs for the whole week 
# then choose a vehicle journey id (at a busy time of day to increase chance it will have stopped at all stops)

# doesn't actually matter for this purpose if we take in more than one run (if a vehicle journey id was reused in 
# the same day)
routes = {}
vehicle_journey_groups = {}

for journey_name, journey_group in df_week:
    vehicle_groups = df_week.get_group(journey_name).groupby(['VehicleJourney_ID'])
    count = 0
    for v_name, v_group in vehicle_groups:
        count += 1
        if count == 10 or count == (len(vehicle_groups)):
            gp = vehicle_groups.get_group(v_name)
            for index, row in gp.iterrows():
                converted_timestamp = convert_timestamp(row['Timestamp'])
                tm, day = converted_timestamp[0], converted_timestamp[1]
                gp.set_value(index, 'Time', tm)
                gp.set_value(index, 'Day', day)
            routes[str(journey_name)] = {"vehicleJourney_ID": str(v_name), "stops": []}
            vehicle_journey_groups[str(journey_name)] = {"vehicleJourneyGroup": gp}
            break
            
#print(journeys_vehicles)
print(len(routes))
# null not removed for some reason - will pop it from the list of journey ids

471


In [14]:
# Now what I have is a dictionary where the key is the journey id and the value is the vehicle journey id and the group for that 
# vehicle journey id
print(len(routes))

471


### Now for the above list of journey & vehicleJourney combos we want to assign a time and day to each row in each group and then order by time and get a list of unique stops visited by each journeyPattern_ID over the week

In [15]:

for journeyPattern in routes:
    vehicle_gp = vehicle_journey_groups[journeyPattern]["vehicleJourneyGroup"]
    vehicle_gp = vehicle_gp.sort_values(by='Time', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
    stops = vehicle_gp['Stop_ID'].unique().tolist()
    routes[journeyPattern]["stops"] = stops
        


In [16]:
print(type(routes))

<class 'dict'>


In [17]:
with open('routes.txt', 'w') as f:
    json.dump(routes, f)

In [18]:
gp.head()

,Timestamp,Line_ID,Direction,JourneyPattern_ID,Time_Frame,VehicleJourney_ID,Operator,Congestion,Longitude,Latitude,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Day,Time
604545,1352373895000000,41,0,null,2012-11-08,40,SL,0,-6.250983,53.459633,0,41007,33110,4843,1,Thursday,11:24:55
601980,1352719478000000,41,0,null,2012-11-12,40,SL,0,-6.251000,53.459599,0,41007,33115,4843,1,Monday,11:24:38


In [19]:
gp.shape

(2, 17)